# Baseball Stats Prediction Project

## Introduction

Perhaps the most quintessential of the major four American sports, baseball appears to be growing in popularity at a rapid pace. I can say this because, as a former baseball player myself, the amount of media coverage being given to the sport has increased substantially since I played about a decade ago.

Whether it be the rise of social media used to market the sport or the creation of a plethora of sportsbook betting apps, fans all over the country (and the world) have the ability to be dialed in 24/7 to America's past time. 

However, with the rise of advanced analytics and the ascension of sports betting to the mainstream, now more than ever, coaches and fans alike are reliant upon statistics to make or break their real (or fantasy) teams. 

This raises the question: "Can we use Machine Learning and Data Science techniques to help predict these all important statistics?". The answer to this question is what this project is all about, specifically when it comes to the Wins Above Replacement (WAR) statistic.

### Preprocessing and Data Exploration

In [1]:
# We then import our required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# We then import py baseball to utilize its dataset
from pybaseball import batting_stats

In [3]:
# We then define the start and end dates for our dataset
START = 2002
END = 2022
# We used this information to filter our dataset when loading it in for the first time

We then remove players who are only in our dataset for a single season. This is because we are trying to make predictions for players based upon a previous season's statistics. Hence a player with only a single year of experience will not qualify for our model.

In [4]:
# Before we filter our data set, we read it into a dataframe and explore
batting = pd.read_csv("batting.csv", index_col=0)

In [5]:
batting.head(5)

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA
0,1109,2002,Barry Bonds,SFG,37,143,403,612,149,70,...,NaN,NaN,NaN,NaN,0,0.127,0.191,NaN,NaN,NaN
1,1109,2004,Barry Bonds,SFG,39,147,373,617,135,60,...,NaN,NaN,NaN,NaN,0,0.124,0.164,NaN,NaN,NaN
3,15640,2022,Aaron Judge,NYY,30,152,554,676,174,85,...,0.268,118.4,241.0,0.61,395,0.170,0.288,NaN,NaN,NaN
15,13611,2018,Mookie Betts,BOS,25,136,520,614,180,96,...,0.131,110.6,217.0,0.50,434,0.220,0.270,NaN,NaN,NaN
2,1109,2003,Barry Bonds,SFG,38,130,390,550,133,65,...,NaN,NaN,NaN,NaN,0,0.135,0.223,NaN,NaN,NaN


We know that there is no way we will be able to incroporate all 320 columns into our model, so we will have to deal with that later. As for now, it looks like the column "IDfg" corresponds to an individual player's name. This will be able to help us filter our dataset. 

In [6]:
# We now filter our dataframe by creating a groupby and filtering out players who only played for a single season 
batting = batting.groupby("IDfg", group_keys=False).filter(lambda x: x.shape[0] > 1) # Lambda function checks to ensure there are 2 seasons of data

### Target Definition

We now define the WAR column as our target column for our Machine Learning model. However, we have to manipulate our dataframe again and create a function where we backfill the WAR of each player from the next season

In [7]:
def next_season(player):
    player = player.sort_values("Season")  # Sort the dataframe on each player's season
    player["Next_WAR"] = player["WAR"].shift(-1)  # Creates a new column where we shift each players WAR back by a single season
    return player

batting = batting.groupby("IDfg", group_keys=False).apply(next_season)

In [8]:
# Let's look to see if our changes worked
batting[["Name", "Season", "WAR", "Next_WAR"]]

,Name,Season,WAR,Next_WAR
5561,Alfredo Amezaga,2006,1.1,2.0
5001,Alfredo Amezaga,2007,2.0,1.2
5251,Alfredo Amezaga,2008,1.2,NaN
1165,Garret Anderson,2002,3.7,5.1
866,Garret Anderson,2003,5.1,0.8
...,...,...,...,...
6034,Owen Miller,2022,0.6,NaN
4880,Andrew Vaughn,2021,-0.3,-0.1
3004,Andrew Vaughn,2022,-0.1,NaN
6619,Ha-seong Kim,2021,0.5,3.2


### Data Cleaning

The bane of every Data Scientist's existence: data cleaning. Recall that we mentioned earlier that we had 319 columns of data and that we would not be incorporating all of them into our feature matrix. This is partly because not all of them are useful, but also because some of them are just too dirty to use.

As a result, we will drop columns that are too corrputed to use from missing values while also isolating columns that have the potential to be cleaned.

In [9]:
# We first get rid of columns that have too many null values
null_count = batting.isnull().sum()  # We first count nulls in each column
null_count

IDfg           0
Season         0
Name           0
Team           0
Age            0
            ... 
CSW%           0
xBA         6751
xSLG        6751
xwOBA       6751
Next_WAR    1178
Length: 320, dtype: int64

In [10]:
complete_cols = list(batting.columns[null_count == 0])  # A list of columns with 0 missing values

In [11]:
print("There are {} complete columns".format(len(complete_cols)))

There are 131 complete columns


The fact that there are 131 complete columns is quite astonishing. This may be enough to construct a model from. For this model iteration, let's just keep these columns, along with our recently created Next_WAR column to build our model from. Next WAR is our target, hence the reason we keep it, even if it has null values.

In [12]:
# Index our original dataframe with our recently collected complete columns
batting = batting[complete_cols + ["Next_WAR"]].copy()

In [13]:
# Check to make sure we have 132 columns
batting.head(5)

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,Events,CStr%,CSW%,Next_WAR
5561,1,2006,Alfredo Amezaga,FLA,28,132,334,378,87,72,...,86,107,113,143,109,63,0,0.188,0.256,2.0
5001,1,2007,Alfredo Amezaga,FLA,29,133,400,448,105,80,...,92,101,112,109,113,75,0,0.175,0.227,1.2
5251,1,2008,Alfredo Amezaga,FLA,30,125,311,337,82,61,...,99,101,101,123,111,64,0,0.178,0.244,NaN
1165,2,2002,Garret Anderson,ANA,30,158,638,678,195,107,...,118,91,80,65,97,129,0,0.137,0.232,5.1
866,2,2003,Garret Anderson,ANA,31,159,638,673,201,119,...,112,101,80,90,99,109,0,0.164,0.252,0.8


The next item on our checklist to check for is the datatype of each column. Recall that the vast majority of ML algorithms only accept numeric datatypes as input. As a result, we need a way to handle non-numeric columns. 

In [14]:
# Check to see how many object datatype columns there are in our dataset
object_columns = [column for column in batting.columns if batting[column].dtype == 'object']

In [15]:
print("There are {} non-numeric columns".format(len(object_columns)))

There are 4 non-numeric columns


We may be tempted to use dummy variables to handle these columns since there are only a few. However, let's take a look at the name of each column in our list.

In [16]:
for c in object_columns:
    print(c)

Name
Team
Dol
Age Rng


In [17]:
# Let's check the values of each column
batting["Dol"]

5561      $5.5
5001     $11.2
5251      $7.2
1165     $14.6
866      $22.0
         ...  
6034      $4.5
4880    ($2.6)
3004    ($1.0)
6619      $4.0
4797     $25.4
Name: Dol, Length: 6751, dtype: object

In [18]:
batting["Age Rng"]  # This is the range of age for each player for each season

5561    28 - 28
5001    29 - 29
5251    30 - 30
1165    30 - 30
866     31 - 31
         ...   
6034    25 - 25
4880    23 - 23
3004    24 - 24
6619    25 - 25
4797    26 - 26
Name: Age Rng, Length: 6751, dtype: object

In [19]:
# Let's drop these two columns as they do not give us any valuable information
batting.drop(['Dol', 'Age Rng'], axis=1, inplace=True)

The best way to handle our Team name column will be to create dummy variables. This essentially converts each team name into a column itself whereby the values of the columns are a boolean to indicate if a player was a member of that respective team

In [20]:
# We first create a new column that is a coded categorical dtype form of Team
batting["team_code"] = batting["Team"].astype("category").cat.codes  # This turns our team name into a set of numbers

In [21]:
# We now copy our dataframe in preparation to drop null rows
batting_full = batting.copy()  # Used to forecast for 2023 season
batting = batting.dropna().copy()

### Feature Selection and Engineering

In this section, we go through each column to choose the best features for our model, and if the situation presents itself, create new features that could be useful in predicting our WAR statistic. 

In [22]:
# We first import packages necessary for training a model
from sklearn.linear_model import Ridge
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit

rr = Ridge(alpha=1)  # Create an instance of Ridge Regression with regularization parameter
split = TimeSeriesSplit(n_splits=3)  # Splits our model to not bleed future information

# We then use the Sequential Feature selector to select the best 20 features of our dataset using our time series split for cross validation
sfs = SequentialFeatureSelector(rr, n_features_to_select=20, direction="forward", cv=split, n_jobs=4)

In [23]:
# We then remove columns that will not be able to be used by our model or can cause overfitting
remove_columns = ["Next_WAR", "Name", "Team", "IDfg", "Season"]  # Next_WAR is our target
selected_columns = batting.columns[~batting.columns.isin(remove_columns)]  # select all columns not in our remove list

When working with a Ridge Regression Model, we must scale our dataset. As a result, we will use the MinMax scale technique

In [24]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
batting.loc[:, selected_columns] = scaler.fit_transform(batting[selected_columns])

In [25]:
# Let's check to make sure our columns are scaled correctly
batting.head()

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,...,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,Events,CStr%,CSW%,Next_WAR,team_code
5561,1,2006,Alfredo Amezaga,FLA,0.346154,0.735043,0.312950,0.307958,0.245690,0.278302,...,0.539326,0.503759,0.662921,0.652174,0.210884,0.0,0.582979,0.524229,2.0,0.352941
5001,1,2007,Alfredo Amezaga,FLA,0.384615,0.743590,0.431655,0.429066,0.323276,0.316038,...,0.471910,0.496241,0.471910,0.710145,0.292517,0.0,0.527660,0.396476,1.2,0.352941
1165,2,2002,Garret Anderson,ANA,0.423077,0.957265,0.859712,0.826990,0.711207,0.443396,...,0.359551,0.255639,0.224719,0.478261,0.659864,0.0,0.365957,0.418502,5.1,0.029412
866,2,2003,Garret Anderson,ANA,0.461538,0.965812,0.859712,0.818339,0.737069,0.500000,...,0.471910,0.255639,0.365169,0.507246,0.523810,0.0,0.480851,0.506608,0.8,0.029412
2572,2,2004,Garret Anderson,ANA,0.500000,0.564103,0.507194,0.475779,0.443966,0.400943,...,0.494382,0.218045,0.297753,0.608696,0.448980,0.0,0.531915,0.585903,-0.2,0.029412


In [26]:
# Nice, let's take a look at some summary statistics
batting.describe()

,IDfg,Season,Age,G,AB,PA,H,1B,2B,3B,...,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,Events,CStr%,CSW%,Next_WAR,team_code
count,5573.000000,5573.000000,5573.000000,5573.000000,5573.000000,5573.000000,5573.000000,5573.000000,5573.000000,5573.000000,...,5573.000000,5573.000000,5573.000000,5573.000000,5573.000000,5573.000000,5573.000000,5573.000000,5573.000000,5573.000000
mean,5363.747712,2011.159699,0.360619,0.652801,0.478718,0.481002,0.366025,0.290521,0.399318,0.103441,...,0.457543,0.403189,0.410895,0.511017,0.478671,0.172905,0.498928,0.545856,1.792661,0.474240
std,5129.239240,5.609925,0.147454,0.255950,0.242497,0.262307,0.182585,0.138789,0.171748,0.105867,...,0.114006,0.131203,0.121094,0.130380,0.134008,0.273866,0.137193,0.120701,1.994039,0.305087
min,1.000000,2002.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.400000,0.000000
25%,1130.000000,2006.000000,0.269231,0.478632,0.275180,0.257785,0.211207,0.179245,0.258621,0.043478,...,0.382022,0.315789,0.331461,0.420290,0.387755,0.000000,0.408511,0.466960,0.300000,0.205882
50%,3531.000000,2011.000000,0.346154,0.709402,0.507194,0.508651,0.370690,0.283019,0.379310,0.086957,...,0.460674,0.398496,0.404494,0.507246,0.489796,0.000000,0.493617,0.546256,1.500000,0.470588
75%,9015.000000,2016.000000,0.461538,0.871795,0.688849,0.711073,0.508621,0.391509,0.517241,0.130435,...,0.528090,0.488722,0.483146,0.594203,0.564626,0.345576,0.591489,0.625551,2.900000,0.735294
max,27506.000000,2021.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,11.900000,1.000000


In [27]:
# We now deploy our feature selection class from earlier
sfs.fit(batting[selected_columns], batting["Next_WAR"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=Ridge(alpha=1), n_features_to_select=20,
                          n_jobs=4)

In [28]:
# We now extract the list of best features
predictors = list(selected_columns[sfs.get_support()])
predictors

['Age',
 'IBB',
 'SO',
 'SB',
 'BU',
 'BABIP',
 'IFH%',
 'WAR',
 'Spd',
 'PH',
 'CB%',
 'O-Contact%',
 'wGDP',
 'Oppo%',
 'OBP+',
 'SLG+',
 'LD+%',
 'Pull%+',
 'Soft%+',
 'Hard%+']

### Model Creation

In this section, we will begin the construction of our model. This is without a doubt my favorite part of any Machine Learning workflow. However, there is one more step we need to take before we can begin constructing our model(s).

Normally, we perform cross validation on our datasets in order to assess the model's accuracy and its ability to overfit/underfit. For this project, we need to figure out a different way to assess our accuracy because we cannot perform cross validation on time-series data. 

As a result, we want to make sure that we use the past data to predict future data. We will account for this with the following function

In [29]:
def backtest(data, model, predictors, start=5, step=1):  # We define our function with associated parameters
    all_predictions = []  # A list to hold our prediction scores (as dataframes)
    
    years = sorted(data["Season"].unique())  # Returns a list of the numbers 2002-2021 from the seasons column
    
    # We then iterate through years 2002-2006 to make 2007 predictions for WAR. We then use '02-'07 to predict '08 and so on.
    for i in range(start, len(years), step):  # Allows us to use historical data to predict future, starting with the 2007 season
        current_year = years[i]  # Extract the acutal year of the season
        
        # Split our training and test set
        train = data[data["Season"] < current_year] # Historical years
        test = data[data["Season"] == current_year] # Prediction year, first time through is 07, then 08
        
        # We then train our model
        model.fit(train[predictors], train["Next_WAR"])
        predictions = model.predict(test[predictors])
        
        # We then turn the predictions into a Series
        predictions = pd.Series(predictions, index=test.index)
        
        combined = pd.concat([test["Next_WAR"], predictions], axis=1) # Concatenate data to have real and predicted columns
        combined.columns = ['Actual', 'Predicted']  # Give it column names
        
        # Append our score to our dataframe
        all_predictions.append(combined)
        
    return pd.concat(all_predictions)        

In [30]:
predictions = backtest(batting, rr, predictors)

In [31]:
# Let's take a look at our results
predictions

,Actual,Predicted
5001,1.2,1.556063
1923,1.4,0.811510
3105,-0.1,0.627641
5793,0.6,0.888410
1105,4.8,2.222727
...,...,...
1913,1.9,2.752788
5872,1.1,1.909345
7027,0.6,1.543650
4880,-0.1,1.800711


### Error Analysis

No Machine Learning project is complete without a way to measure our model's accuracy. Hence we need to find a summary statistic that will be able to quantify our model's ability to predict.

In [32]:
# We will use an RMSE score to measure the accuracy
from sklearn.metrics import mean_squared_error
score = np.sqrt(mean_squared_error(predictions['Actual'], predictions['Predicted']))
print(score)

1.6701577709051214


### Model Improvement

As it stands currently, our model is not taking into consideration the trend of an individual player. For example, our model is not noticing if a player is progressively getting better (or worse) as his career progresses. 

If we can figure out a way to implement this into our model, we should see a good increase in our model's RMSE score.

In [33]:
# We create a function to help identify trends in our model
def player_history(df):
    df = df.sort_values("Season") # Sort our df on Seasons
    df["player_season"] = range(0, df.shape[0])  # Create a new column
    
    # We then create a new column by using the expand method on the seasons of each player and see what the correlation for the player season and their WAR is
    # The logic for this column is a bit complex, but a lot of it is designed to format our column appropriately
    df["war_corr"] = list(df[["player_season", "WAR"]].expanding().corr().loc[(slice(None), "player_season"), "WAR"])
    df["war_corr"].fillna(1, inplace=True)  # Fills 1st season with 1:1 correlation
    
    # We then create a column that compares the difference between the previous season war and the current season
    df["war_diff"] = df["WAR"] / df["WAR"].shift(1)  # Takes WAR of current season and divides it by previous season
    df["war_diff"].fillna(1, inplace=True)
    
    df["war_diff"][df["war_diff"] == np.inf] = 1 # Replace infinite values with 1
    
    return df

In [34]:
# We then apply our new function
batting = batting.groupby("IDfg", group_keys=False).apply(player_history)  # For each player, call player history and pass in data

In [35]:
# We then create a function to find averages
def group_averages(df):
    return df["WAR"] / df["WAR"].mean()

In [36]:
batting["war_season"] = batting.groupby("Season", group_keys=False).apply(group_averages)

In [37]:
# We create a new list to include these newly created features
new_predictors = predictors + ["player_season", "war_corr", "war_season", "war_diff"]

In [38]:
# We now call the same function we created before
predictions2 = backtest(batting, rr, new_predictors)
score2 = np.sqrt(mean_squared_error(predictions2["Actual"], predictions2["Predicted"]))

In [39]:
# We now score our model results again
print("After improving our model, our new RMSE score is {}".format(score2))

After improving our model, our new RMSE score is 1.6424244525797052


### Diagnosing issues with our model

In [40]:
# Let's take a look at the coefficients of our ridge regression model
coefficients = pd.Series(rr.coef_, index=new_predictors)

In [41]:
coefficients.sort_values()

Age             -2.616844
BABIP           -1.703344
WAR             -1.670470
SLG+            -1.478175
Soft%+          -1.322185
BU              -0.933384
PH              -0.725718
SO              -0.697091
war_diff        -0.584005
wGDP            -0.419768
CB%             -0.289126
LD+%            -0.236270
Pull%+          -0.171574
war_corr        -0.126820
player_season   -0.007801
O-Contact%       0.268213
IFH%             0.363196
OBP+             0.490683
Oppo%            0.731710
Spd              0.739128
SB               1.030106
IBB              1.647586
Hard%+           2.360402
war_season       3.411393
dtype: float64

In [42]:
# Let's look at the difference between our acuta and predicted values
diff = predictions2["Actual"] - predictions2["Predicted"]
merged = predictions2.merge(batting, left_index=True, right_index=True)
merged["diff"] = (diff).abs()
merged[["IDfg", "Season", "Name", "WAR", "Next_WAR", "diff"]].sort_values(["diff"])

,IDfg,Season,Name,WAR,Next_WAR,diff
3901,5227,2013,Jon Jay,0.322981,1.7,0.000417
1698,9368,2021,Evan Longoria,0.316770,1.6,0.000579
3005,1825,2012,David DeJesus,0.322981,2.0,0.000932
3096,10047,2021,Wil Myers,0.335404,0.7,0.001656
6828,2179,2009,Ronny Cedeno,0.167702,0.8,0.001911
...,...,...,...,...,...,...
3163,4810,2007,Brian McCann,0.304348,8.6,6.332956
3249,5631,2010,Matt Kemp,0.211180,8.3,6.339793
873,9166,2010,Buster Posey,0.459627,10.1,6.623580
451,15640,2021,Aaron Judge,0.552795,11.1,7.172556


## Conclusion

After a lot of hard work, we were able to create a Ridge Regression model that was able to predict a player's WAR with a fair amount of accuracy. We were able to select the 20 best features to predict our model and were able to engineer a couple of new features to decrease our overall RMSE score.

With that being said, our model is far from perfect. As the table above shows, for some reason our model is very poor in predicting the WAR of some star players early in their careers. This is something we should address when engineering our features to improve our model's accuracy even further.